# Module 19: Filtering a Nuisance Covariance

### Nuisance Covariates

Model factors associated with known sources of variability, but that are <i>not</i> related to the experimental hypothesis, need to be included in the GLM. 

For example, these "nuisance regressors" could be signal drift, physiological artifacts (e.g. respiration), and head motion (e.g. six regressors comprising of three translations and three rotations. And sometimes tranformations of the six regressors might also be included.

#### Drift

Drift is slow changes in voxel intensity over time due to scanner instabilities. We need to include drift parameters in our models which usually take the form of splines, polynomial basis sets or discrete cosine basis sets. 

In the example here our design matrix has a first column that is kind of just boxes which correspond to the task in the study. The next column is a baseline, and the all the others correspond to a discrete cosine basis set, which hopefully models the drift present in the data <img src='discos.png'>

We can look at the relative contribution of these parts in modeling the actual data (blue). The Red uses only the predicted response (task data), which clearly doesn't catch any drift. The red stays horizontal as the blue line drops over time. The black line is just the mean of the baseline and the drift components over time, so it does not spike like the others. The green line is the predicted response while accounting for drift, and this gets us much closer to the actual data. It has the peaks like the red line, but it drops over time like the black line.
<img src='highpass.png'>

<b>Note</b>: We want to remove the black line, which is essentially noise due to drift, and get to the red line which is the signal of interest.

### Transient Gradient Artifacts

Sometimes we get spikes in the data during acquisition, which can appear as very bright bands in constructed brain image:
<img src='tga.png'>

In essence, these spikes are outliers in the signal. To account for them we can do outlier detection by looking at the successive differences in the frequency (Root Mean Squared of the Success Differences) which we can see in the above image. We can then include these spikes as covariates in the design matrix with one regressor per "bad" image. As seen below, this uses 1 degree of freedom to mop up some of this variance: <img src='nuis.png'>

#### Physiological Noise

Respiration and heart rate give rise to periodic noise that is often aliased into task frequencies. Keep in mind that the sampling rate must be at least twice the frequency of the physiological noise curve if we want to model it. Often, the TR is too low to do this. 

<b>Note</b>: that this type of noise is hard to model and is often left in the data, giving rise to autocorrelations.

However, this noise can be measured in other ways by measuring neuronal activation, the respiration cycle, and the cardiac cycle and retrospective image correction (RETROICOR) can be accomplished. We can also use respiration volume (CO2) and heart rate to account for this noise in a process called RVHRCOR. 
<img src='retroicor.png'>

#### Head Motion

Everybody moves their head a little during runs. Basic motion correction (image realignment) is done in the pre-processing stage, but motion creates complex changes in the magnetic field, and "spin history" artifacts are created which cannot be removed.

There are two basic approaches for dealing with head movement. 
<ul>
<li>Nuisance Regressors: Movement and global CSF-related covariates
<li>Scrubbing: Drop images with high movement estimates (treat data as missing))
</ul>

When using nuisance regressors, we add components to the design matrix. We're trying to model not just the linear movements, but also the squares of the movements, their successive differences (derivative), and the squares of their successive differences. So in the case below, for every run 24 additional movement parameter covariates are included.
<img src='headmove.png'>

<b>Example</b>: in an experiment, participants were given either shock cues or safe cues during shock conditioning. In a group analysis with a sample size of 25, we would hope for the problems to average out. But this study actually ended up finding activation patterns in places like the ventricles, which is physiologically implausible. 

Also when we look at histograms of the contrast values within each brain, we see that wildly different results across brains occur. Some show whole brain deactivation and some show whole brain activation, but we would expect the average to be 0 and the scales for all participants should be the same. 

We see this assumption is not met in multiple participants here:
<img src='problems.png'>

But if we take our design matrix and add covariates for both motion and image spikes we get a new design matrix that looks like this: 
<img src='newdesign.png'>

And afterwards, we see that our histograms are much cleaner between subjects. There are still some problems though. Most are centered on 0, and the distributions are much closer to the same scale, but that's the noise we have to live with:
<img src='newhisto.png'>

Now when the group analysis is performed, we see phenomena that are both physiologically possible and that align with previously done studies. <img src='newgroup.png'>